In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable

In [2]:
# Training settings
batch_size=64
test_batch_size=1000
epochs=5
lr=0.01
momentum=0.5
no_cuda=False,
seed=42 #help='random seed (default: 1)')
log_interval=100 #how many batches to wait before logging training status

cuda = not no_cuda and torch.cuda.is_available()

In [3]:
torch.manual_seed(seed)
if cuda:
    torch.cuda.manual_seed(seed)

Params for if using cuda

In [4]:
params = {'num_workers': 1, 'pin_memory': True} if cuda else {}

#### Loading MNIST data and normalizing it

In [5]:
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=batch_size, shuffle=True, **params)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=batch_size, shuffle=True, **params)

Processing...
Done!


#### Creating the model

nn.Module = Base class for all neural network modules.

2d conv = nn.Conv2d(filtersize_input,filters_out, kernel_size)   #padding??  
fully connected = nn.Linear(inputs,outputs) #Applies a linear transformation to the incoming data: y=Ax+b  Linear because no activation ala relu  
dropout = nn.Dropout(p=0.5, inplace=False)  

# what is the Functional api ?

In [6]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(784, 384) # we define in and out
        self.fc2 = nn.Linear(384, 100)
        self.fc3 = nn.Linear(100, 10)

    def forward(self, x):
        x = x.view(-1, 784) #reshaping the input to flatten
        x = F.relu(self.fc1(x)) # wraping the relu in here
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        #x = F.dropout(x, training=self.training)
        #x = self.fc2(x)
        return F.log_softmax(x)

model = Net()

In [7]:
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum)

In [8]:
def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        if cuda:
            data, target = data.cuda(), target.cuda()
        data, target = Variable(data), Variable(target)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.data[0]))

In [9]:
def test(epoch):
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        if cuda:
            data, target = data.cuda(), target.cuda()
        data, target = Variable(data, volatile=True), Variable(target)
        output = model(data)
        test_loss += F.nll_loss(output, target).data[0]
        pred = output.data.max(1)[1] # get the index of the max log-probability
        correct += pred.eq(target.data).cpu().sum()

    test_loss = test_loss
    test_loss /= len(test_loader) # loss function already averages over batch size
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))



In [10]:
for epoch in range(1, epochs + 1):
    train(epoch)
    test(epoch)

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.305221
Train Epoch: 1 [6400/60000 (11%)]	Loss: 1.374701
Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.787756
Train Epoch: 1 [19200/60000 (32%)]	Loss: 0.807238
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.736443
Train Epoch: 1 [32000/60000 (53%)]	Loss: 0.474307
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.721811
Train Epoch: 1 [44800/60000 (75%)]	Loss: 0.590986
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.584602
Train Epoch: 1 [57600/60000 (96%)]	Loss: 0.461196

Test set: Average loss: 0.4861, Accuracy: 8366/10000 (83.66%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.510801
Train Epoch: 2 [6400/60000 (11%)]	Loss: 0.578975
Train Epoch: 2 [12800/60000 (21%)]	Loss: 0.705920
Train Epoch: 2 [19200/60000 (32%)]	Loss: 0.431729
Train Epoch: 2 [25600/60000 (43%)]	Loss: 0.327314
Train Epoch: 2 [32000/60000 (53%)]	Loss: 0.383482
Train Epoch: 2 [38400/60000 (64%)]	Loss: 0.528773
Train Epoch: 2 [44800/60000 (75%)]	Loss: 0.500074
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.48117

KeyboardInterrupt: 